<a href="https://colab.research.google.com/github/sophiewagner7/its-too-nice-out-to-take-a-cab/blob/main/notebooks/pre_process_weather_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install metar-taf-parser-mivek

In [ ]:
import pandas as pd
#from metar import Metar
#from metar_taf_parser.parser.parser import MetarParser



In [ ]:
#obs = Metar.Metar('METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T02270215')
#print(obs.string())

In [ ]:
root_url = "https://github.com/sophiewagner7/nyc-weather/raw/refs/heads/main/data/GHCNh"

dfs = []
for year in range(2019, 2025):
  file_name = f"GHCNh_USW00094728_{year}.parquet"
  df_fragment = pd.read_parquet(f"{root_url}/{file_name}")
  dfs.append(df_fragment)

df = pd.concat(dfs)

columns = [
    "DATE",
    "temperature",
    "dew_point_temperature",
    "station_level_pressure",
    "wind_direction",
    "wind_speed",
    "wind_gust",
    "precipitation",
    "relative_humidity",
    "wet_bulb_temperature",
    "pres_wx_MW1",
    "pres_wx_MW2",
    "pres_wx_MW3",
    "snow_depth",
    "visibility",
    "altimeter",
    "sky_cover_1",
    "sky_cover_2",
    "sky_cover_3",
    "remarks"
]
df = df[columns]

df["datetime"] = pd.to_datetime(df.DATE)
df = df[df.datetime < "2024-06-26"]

df["date"] = df.datetime.dt.date
df["hour"] = df.datetime.dt.hour

df.to_parquet("GHCNh_USW00094728_2019_to_2024.parquet")
